In [18]:
import lightgbm as lgb
from data_pipeline import data_transform_pipeline, cap_sales, data_transform_pipeline_2, apply_exp, apply_ln
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import re

data = pd.read_csv('data/train.csv')

train_data = data[data['date'] < '2017-08-01']
train_data = cap_sales(train_data)
test_data = data[data['date'] >= '2017-08-01']
test_data = cap_sales(test_data)

X_train = train_data
y_train = X_train.pop('sales')

X_test = test_data
y_test = X_test.pop('sales')

X_train = data_transform_pipeline.fit_transform(X_train)
X_test = data_transform_pipeline.transform(X_test)

X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# y_train = apply_ln(y_train)
# y_test = apply_ln(y_test)



cv_split = TimeSeriesSplit(n_splits=4, test_size=2000)
model = lgb.LGBMRegressor(force_row_wise=True, n_estimators=500, learning_rate=0.05, num_leaves=31, subsample=0.8, colsample_bytree=0.8)

c:\Users\tzbk\CODIGOS\python\machine_learning\time_series_ml_analysis\data_pipeline.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sales = df.sales.map(lambda x: x if x < cap else cap)


In [2]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=cv_split, scoring='neg_mean_squared_error')

[LightGBM] [Info] Total Bins 814
[LightGBM] [Info] Number of data points in the train set: 2966158, number of used features: 40
[LightGBM] [Info] Start training from score 2.917942
[LightGBM] [Info] Total Bins 814
[LightGBM] [Info] Number of data points in the train set: 2968158, number of used features: 40
[LightGBM] [Info] Start training from score 2.918328
[LightGBM] [Info] Total Bins 815
[LightGBM] [Info] Number of data points in the train set: 2970158, number of used features: 40
[LightGBM] [Info] Start training from score 2.918916
[LightGBM] [Info] Total Bins 816
[LightGBM] [Info] Number of data points in the train set: 2972158, number of used features: 40
[LightGBM] [Info] Start training from score 2.919558


In [19]:
model.fit(X_train, y_train)

[LightGBM] [Info] Total Bins 817
[LightGBM] [Info] Number of data points in the train set: 2974158, number of used features: 40
[LightGBM] [Info] Start training from score 356.541586


LGBMRegressor(colsample_bytree=0.8, force_row_wise=True, learning_rate=0.05,
              n_estimators=500, subsample=0.8)

In [20]:
predictions = model.predict(X_test)
# predictions = np.e **predictions + 1
# y_test = np.e ** y_test  + 1
print(f"MSE: {mean_squared_error(y_test, predictions)}")
print(f"ROOT of MSE: {mean_squared_error(y_test, predictions)**0.5}")

MSE: 929476.1238286097
ROOT of MSE: 964.0934206956346
